In [2]:
import mne
import os
import scipy.io
import listen_italian_functions
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import pickle
import warnings
warnings.filterwarnings('ignore')
from tqdm import tnrange, tqdm_notebook
from itertools import combinations

data_path = os.path.dirname(os.path.dirname(os.getcwd()))

subject_name = ['Alice','Andrea','Daniel','Elena','Elenora','Elisa','Federica','Francesca','Gianluca1','Giada','Giorgia',
                'Jonluca','Laura','Leonardo','Linda','Lucrezia','Manu','Marco','Martina','Pagani','Pasquale','Sara',
                'Silvia','Silvia2','Tommaso']

remove_first = 0.5 #seconds


# Read the epoches

In [3]:
Tmin = 0
Tmax = 3.51
trial_len = 2

GA_epoches = []
for s in subject_name:
    save_path = data_path + '/python/data/coherence_epochs/'+s+'-coh-epo-'+str(Tmin)+'-' \
    +str(Tmax)+'-trialLen-'+str(trial_len)+'.fif'
    epochs = mne.read_epochs(save_path)
    GA_epoches.append(epochs)
    print('----------------------------------------------------------------------------------------------------------------'+s)


This filename (C:\Users\SMukherjee\Desktop\projects\listen_italian_motor_entrainment\analysis/python/data/coherence_epochs/Alice-coh-epo-0-3.51-trialLen-2.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif or -epo.fif.gz
Reading C:\Users\SMukherjee\Desktop\projects\listen_italian_motor_entrainment\analysis/python/data/coherence_epochs/Alice-coh-epo-0-3.51-trialLen-2.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    3507.50 ms
        0 CTF compensation matrices available
124 matching events found
No baseline correction applied
124 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
----------------------------------------------------------------------------------------------------------------Alice
This filename (C:\Users\SMukherjee\Desktop\projects\listen_italian_motor_entrainment\analysis/python/data/coherence_epochs/Andrea-coh-epo-0-3.51-trialLen-2.fif) doe

121 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
----------------------------------------------------------------------------------------------------------------Giada
This filename (C:\Users\SMukherjee\Desktop\projects\listen_italian_motor_entrainment\analysis/python/data/coherence_epochs/Giorgia-coh-epo-0-3.51-trialLen-2.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif or -epo.fif.gz
Reading C:\Users\SMukherjee\Desktop\projects\listen_italian_motor_entrainment\analysis/python/data/coherence_epochs/Giorgia-coh-epo-0-3.51-trialLen-2.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    3507.50 ms
        0 CTF compensation matrices available
121 matching events found
No baseline correction applied
121 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
-----------------------------------------------------

Reading C:\Users\SMukherjee\Desktop\projects\listen_italian_motor_entrainment\analysis/python/data/coherence_epochs/Pagani-coh-epo-0-3.51-trialLen-2.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    3507.50 ms
        0 CTF compensation matrices available
122 matching events found
No baseline correction applied
122 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
----------------------------------------------------------------------------------------------------------------Pagani
This filename (C:\Users\SMukherjee\Desktop\projects\listen_italian_motor_entrainment\analysis/python/data/coherence_epochs/Pasquale-coh-epo-0-3.51-trialLen-2.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif or -epo.fif.gz
Reading C:\Users\SMukherjee\Desktop\projects\listen_italian_motor_entrainment\analysis/python/data/coherence_epochs/Pasquale-coh-epo-0-3.51-trialLen-2.fif ...


# bandpass filter epoches and save as fieldtrip format for partial coherence

In [ ]:
iter_freqs = [
    ('fr', 1, 3),
    #('fr', 2, 4),
    #('fr', 3, 5),
    ('fr', 4, 6)
    #('fr', 5, 7),
    #('fr', 6, 8),
    #('fr', 7, 9),
    #('fr', 8, 10),
    #('fr', 9, 11),
    #('fr', 10, 12)
]

save_path = data_path +'/python/data/partialCoh/partialCoh-trailLen-' +str(trial_len)+'-removedFirst-'+ str(remove_first)

features = ['envelop','jawaopening','lipaparature','lipProtrusion','TTCD','TMCD','TBCD']
condition = ['Hyper','Normal','Hypo','All']
delay = np.arange(0,1.1,0.1)


extra_channels = ['envelop','jawaopening','lipaparature','lipProtrusion','TTCD','TMCD','TBCD']
eeg_channles = np.setdiff1d(GA_epoches[0].ch_names, extra_channels)
event_id = {'Hyper': 1,'Normal': 2,'Hypo': 3}
ch_types = np.repeat('eeg', len(features)+59)
ch_names = np.hstack((eeg_channles,features))        
info = mne.create_info(ch_names = ch_names.tolist(),ch_types = ch_types,sfreq = GA_epoches[0].info['sfreq'])
ch_names = np.setdiff1d(extra_channels,features)
        
for s in range(0,len(subject_name)):
    for d in delay:   
        for c in condition:
            for fr in range(0,len(iter_freqs)):
                b = str(iter_freqs[fr][0])+ '-'+str(iter_freqs[fr][1])+'-'+str(iter_freqs[fr][2])+'Hz'

                fmin = iter_freqs[fr][1]
                fmax = iter_freqs[fr][2]
                a = listen_italian_functions.partialCoherence_preprocess_delay(GA_epoches[s],remove_first,d,
                                                                               trial_len,extra_channels,eeg_channles,
                                                                               info,ch_names,event_id,fmin,fmax,c)   
    
                d = d.round(decimals=1)
                scipy.io.savemat(save_path+'s-'+'condition-'+c+'-delay-'+str(d)+'-'+b+'-' +subject_name[s],a)
        
        

# remove contribution of lipaperature from the envelop delayed version (partial coherence implemented here) 

coherence
<img src="http://nipy.org/nitime/_images/math/8b2ad8f5230194bf1f946cdde75e57e513e8fb18.png">

partial coherence
<img src="http://nipy.org/nitime/_images/math/6f19a896ad7872e2077078e1d5dc1bac04b2049c.png">

In [4]:
def get_coherence(epochs,sfreq,fmin,fmax,indices):
    con, freqs, times, n_epochs, n_tapers = mne.connectivity.spectral_connectivity(epochs, method='coh',mode='multitaper', 
                                                                                   sfreq=sfreq, 
                                                              fmin=fmin, fmax=fmax,indices=indices,faverage=True, 
                                                              tmin=0, mt_adaptive=False, block_size=1000,verbose='ERROR')

    return con

def partialCoherence_preprocess_delay(epochs,remove_first,d,trial_len,feat,eeg_channles,keep_feat,condition):	

    if condition != 'All':
        E = epochs[condition].copy()
    else:
        E = epochs.copy()
    
    eeg = E.copy().drop_channels(feat)
    speech = E.copy().drop_channels(eeg_channles)
    
    a = np.setdiff1d(feat, keep_feat)
    speech.drop_channels(a)

    E = eeg.copy().crop(d+remove_first,d+remove_first+trial_len)
    S = speech.copy().crop(0.5+remove_first,0.5+remove_first+trial_len)

    c = np.concatenate((E.get_data(),S.get_data()),axis=1)   
        
    return c


In [37]:
features = ['envelop','jawaopening','lipaparature','lipProtrusion','TTCD','TMCD','TBCD']
remove_first = 0.5 #seconds
channel_names = GA_epoches[0].ch_names
eeg_chan = np.setdiff1d(channel_names, features)
condition = 'All'

iter_freqs = [
    ('fr', 1, 3),
    ('fr', 4, 6)
]

keep_feat = ['envelop']
FD = np.arange(-5,6) / 10

remove_feat = ['lipaparature']
RD = np.arange(-5,6) / 10

feat_comb = (['envelop','lipaparature'],['lipaparature','envelop'])
fmin = []
fmax = []
for fr in range(0,len(iter_freqs)):
    fmin.append(iter_freqs[fr][1])
    fmax.append(iter_freqs[fr][2])

In [38]:
frame = [] * len(subject_name) * len(feat_comb) * len(FD) * len(RD) * len(iter_freqs)
for s in range(0,len(subject_name)):
    for f in range(0,len(feat_comb)):
        keep_feat = feat_comb[f][0]
        remove_feat = feat_comb[f][1]
        a=np.where(np.array(channel_names)==keep_feat)[0][0]
        indicesXY = (np.repeat(a,59),np.arange(0,59))
        b=np.where(np.array(channel_names)==remove_feat)[0][0]
        indicesXR = (np.repeat(b,59),np.arange(0,59))
        indicesRY = (np.repeat(a,1),np.repeat(b,1))

        for fd in FD:
            for rd in RD:
                envelop = partialCoherence_preprocess_delay(GA_epoches[s],remove_first,fd + 0.5,trial_len,features,
                                                            eeg_chan,keep_feat,condition)
                lipaparature = partialCoherence_preprocess_delay(GA_epoches[s],remove_first,rd + 0.5,trial_len,features,
                                                                 eeg_chan,remove_feat,condition)                

                conXY = get_coherence(envelop,400,fmin,fmax,indicesXY)
                conXR = get_coherence(lipaparature,400,fmin,fmax,indicesXR)

                c = np.dstack((envelop[:,59,:],lipaparature[:,59,:]))
                c = np.swapaxes(c,1,2)
                conRY = get_coherence(c,400,fmin,fmax,indicesRY)[0]

                for fr in range(0,len(iter_freqs)):
                    b = str(iter_freqs[fr][0])+ ' '+str(iter_freqs[fr][1])+' - '+str(iter_freqs[fr][2])+'Hz'
                    partial_coh_XY_R=[]
                    for i in range(59):
                        a = (abs(conXY[i,fr]-conXR[i,fr]*conRY[fr])**2) / ((1-abs(conXR[i,fr])**2)*(1-abs(conRY[fr])**2))
                        partial_coh_XY_R.append(a)

                    partial_coh_XY_R = np.asarray(partial_coh_XY_R)                        
                    df = pd.DataFrame({'Feature':keep_feat,'FeatureDelay':fd,'RemoveFeature':remove_feat,
                                       'RemoveFeatureDelay':rd,'Freq':b,'Condition':condition,
                                       'Subject': subject_name[s], 'Data':[partial_coh_XY_R]})

                    frame.append(df)
                print(str(fd)+'-'+str(rd)+'-'+subject_name[s])

data=pd.concat((frame),axis=0)
save_path = data_path + '/python/data/partialCoh/Delayed_partialCoh-removedFirst-'+str(remove_first)+'.pkl'
data.to_pickle(save_path)

-0.5--0.5-Alice
-0.5--0.4-Alice
-0.5--0.3-Alice
-0.5--0.2-Alice


KeyboardInterrupt: 